In [2]:
import pandas as pd

df = pd.read_csv('trump_data/trumptweets.csv')
print(df.head())

           id                                               link  \
0  1698308935  https://twitter.com/realDonaldTrump/status/169...   
1  1701461182  https://twitter.com/realDonaldTrump/status/170...   
2  1737479987  https://twitter.com/realDonaldTrump/status/173...   
3  1741160716  https://twitter.com/realDonaldTrump/status/174...   
4  1773561338  https://twitter.com/realDonaldTrump/status/177...   

                                             content                 date  \
0  Be sure to tune in and watch Donald Trump on L...  2009-05-04 20:54:25   
1  Donald Trump will be appearing on The View tom...  2009-05-05 03:00:10   
2  Donald Trump reads Top Ten Financial Tips on L...  2009-05-08 15:38:08   
3  New Blog Post: Celebrity Apprentice Finale and...  2009-05-08 22:40:15   
4  "My persona will never be that of a wallflower...  2009-05-12 16:07:28   

   retweets  favorites mentions hashtags  geo  
0       500        868      NaN      NaN  NaN  
1        33        273      NaN 

In [3]:
print(df.columns)

Index(['id', 'link', 'content', 'date', 'retweets', 'favorites', 'mentions',
       'hashtags', 'geo'],
      dtype='object')


In [4]:
max_retweet_row = df.loc[df['retweets'].idxmax()]

# Afficher la ligne
print(max_retweet_row)

id                                          881503147168071680
link         https://twitter.com/realDonaldTrump/status/881...
content         # FraudNewsCNN # FNNpic.twitter.com/WYUnHjjUjg
date                                       2017-07-02 15:21:42
retweets                                                309892
favorites                                               528265
mentions                                                   NaN
hashtags                                                   # #
geo                                                        NaN
Name: 32059, dtype: object
